1202 개인 공부

## 와이드 & 딥 신경망 (Wide & Deep Neural Network)
- 헝쯔 청, 2016
- 입력의 일부 또는 전체가 출력 층에 바로 연결되는 구조
- 깊은 층: 복잡한 패턴 학습
- 낮은 층: 간단한 규칙 학습
- MLP는 네트워트 모든 층에 데이터를 통과시켜야 한다는 특징이 있음
-> 데이터의 간단한 패턴이 연속된 변환으로 왜곡될 수 있다는 단점
<br>

**입력층 -> 정규화 -> 은닉층 -> … -> 은닉층 -> 층 연결 -> 출력층**
- 와이드 & 딥 신경망 구조에서는 정규화 -> 층 연결로 바로 이동할 수 있는 연결이 추가로 있음.

### Wide(기억) & Deep(추론/일반화) Neural Network
- 내부적으로 기억 & 추론을 동시에 하는 모델을 만들 수 있음
- 시각화 및 해석용으로 출력을 두 개 이상 만들 수 있음


In [ ]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

normalization_layer = tf.keras.layers.Normalization()
hidden_layer1 = tf.keras.layers.Dense(30, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1)

input_ = tf.keras.layers.Input(shape=X_train.shape[1: ])      # input_ : 파이썬 내장함수input()과 이름 충돌을 방지하기 위해 언더바(_) 사용

normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)
model = tf.keras.Model(inputs=[input_], outputs=[output])

**심볼릭 입력: normalization_layer(input_)**
- 각 층을 함수처럼 이용하여 객체를 전달함. (함수형 API라고 불리는 이유)
- 이 때 실제로 데이터가 처리되는 것이 아니라, 케라스에 층을 연결하는 것뿐이므로 심볼릭 입력이라고 함.
<br>

**concat_layer([normalized, hidden2])**
- 층을 순차적으로 연결하는 것이 아닌, normalized 층과 hidden2 두 층을 모두 concat 층에 연결함으로써 wide & deep 신경망 구축

In [ ]:
model.layers

[<InputLayer name=input_layer, built=True>,
 <Normalization name=normalization_1, built=True>,
 <Dense name=dense_1, built=True>,
 <Dense name=dense_2, built=True>,
 <Concatenate name=concatenate, built=True>,
 <Dense name=dense_3, built=True>]

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)     # 적응형 학습률
#optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)     # 고정형 학습률
model.compile(loss="mse",
              optimizer=optimizer,
              metrics=["RootMeanSquaredError"])

In [ ]:
normalization_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test, rmse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - RootMeanSquaredError: 1.4524 - loss: 2.2515 - val_RootMeanSquaredError: 0.7462 - val_loss: 0.5569
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - RootMeanSquaredError: 0.7086 - loss: 0.5032 - val_RootMeanSquaredError: 1.3269 - val_loss: 1.7607
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6252 - loss: 0.3912 - val_RootMeanSquaredError: 1.4183 - val_loss: 2.0116
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6119 - loss: 0.3750 - val_RootMeanSquaredError: 1.9289 - val_loss: 3.7205
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6566 - loss: 0.4333 - val_RootMeanSquaredError: 0.8485 - val_loss: 0.7200
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6101 - loss: 0.3726 - val_RootMeanSquaredError: 1.2992 - val_loss: 1.6878
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5875 - los

**adapt(훈련데이터에 모델을 적응)시키는 이유**: 데이터 정규화를 위해 훈련 데이터의 평균과 분산을 계산
1. 데이터 스케일 통일
2. 데이터 누수 방지: `X_test`, `X_valid의` 정보가 훈련 과정으로 새어들어가는 것을 방지함.
3. 일관된 분산 및 평균 적용: `X_train`의 평균 및 분산을 `X_test`, `X_valid`에도 적용하여 예측
- 이 때 훈련 데이터가 전체 데이터 분포를 충분히 대표한다고 가정함.

In [ ]:
y_pred

array([[0.5762739],
       [1.4627191],
       [4.663391 ]], dtype=float32)

In [ ]:
y_test[:3]

array([0.477  , 0.458  , 5.00001])

예측 오차가 큼(y_pred, y_test) - 하이퍼파라미터 개선이 필요

In [ ]:
input_wide = tf.keras.layers.Input(shape=[5])
input_deep = tf.keras.layers.Input(shape=[6])

norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()

norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)   # 함수형 API를 연결하는 법
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)

model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

정규화 층은 선언과 동시에 함수형 api를 연결할 수 없음. (인스턴스화 불가)
- 이유: 모델 훈련 전, normalization 층에서 adapt() 메서드를 호출할 수 있도록(이 층을 참조할 수 있도록) 해야하기 때문

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)
history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20,
                    validation_data=((X_valid_wide, X_valid_deep), y_valid))
mse_test = model.evaluate((X_test_wide, X_test_deep), y_test)
y_pred = model.predict((X_new_wide, X_new_deep))
print(y_pred)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - RootMeanSquaredError: 1.6443 - loss: 2.7795 - val_RootMeanSquaredError: 1.2478 - val_loss: 1.5571
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.8199 - loss: 0.6732 - val_RootMeanSquaredError: 0.7130 - val_loss: 0.5083
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.7073 - loss: 0.5004 - val_RootMeanSquaredError: 1.0404 - val_loss: 1.0825
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.6541 - loss: 0.4281 - val_RootMeanSquaredError: 0.9088 - val_loss: 0.8260
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - RootMeanSquaredError: 0.6210 - loss: 0.3858 - val_RootMeanSquaredError: 0.7091 - val_loss: 0.5029
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6067 - loss: 0.3682 - val_RootMeanSquaredError: 0.6159 - val_loss: 0.3794
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.6084 - los

In [ ]:
y_test[:3]

array([0.477  , 0.458  , 5.00001])

성능이 더 나빠짐... Wide & Deep Neural Network의 사용이 해당 모델에 적절할지 고려해 봐야 함.
물론 하이퍼파리미터 튜닝의 문제일 수도 있지만?

In [ ]:
# 여러 개의 출력을 생성하는 법
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)      # 은닉층2으로부터 보조 출력을 생성
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output, aux_output])

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1), optimizer=optimizer, metrics=["RootMeanSquaredError", "RootMeanSquaredError"])

- 각 출력은 자신만의 손실함수가 필요함. `loss=("mse", "mse")` 또는 `loss={"output":"mse", "aux_output":"mse"}`
- 하나만 전달해도 되는데 그럼 모든 출력에 하나의 손실함수를 사용함.
- 가중치도 각각 정할 수 있음. (총 합이 1일 필요는 X)
- 이 경우, 보조 출력보다 주 출력의 손실함수를 줄이는 방식으로 최적화함 = **역전파 과정에서 주 출력의 손실을 줄이는** 방향으로 가중치를 수정한다는 의미

In [ ]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid))
)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - dense_11_RootMeanSquaredError: 0.9692 - dense_11_loss: 1.0291 - dense_12_RootMeanSquaredError: 2.0141 - dense_12_loss: 4.3196 - loss: 1.3582 - val_dense_11_RootMeanSquaredError: 0.5985 - val_dense_11_loss: 0.3581 - val_dense_12_RootMeanSquaredError: 1.3977 - val_dense_12_loss: 1.9528 - val_loss: 0.5177
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - dense_11_RootMeanSquaredError: 0.6085 - dense_11_loss: 0.3704 - dense_12_RootMeanSquaredError: 0.7379 - dense_12_loss: 0.5448 - loss: 0.3878 - val_dense_11_RootMeanSquaredError: 0.8079 - val_dense_11_loss: 0.6524 - val_dense_12_RootMeanSquaredError: 1.1374 - val_dense_12_loss: 1.2931 - val_loss: 0.7168
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - dense_11_RootMeanSquaredError: 0.5848 - dense_11_loss: 0.3421 - dense_12_RootMeanSquaredError: 0.6865 - dense_12_loss: 0.4716 - loss: 0.3550 - val_dense_11_RootMeanSquaredError: 0.6703 - val_dense_11_loss: 0.4492 - val_dense_12_R

In [ ]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

print("Main loss:", main_loss)
print("Aux loss:", aux_loss)
print("Main RMSE:", main_rmse)
print("Aux RMSE:", aux_rmse)

162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - dense_11_RootMeanSquaredError: 0.5626 - dense_11_loss: 0.3166 - dense_12_RootMeanSquaredError: 0.6133 - dense_12_loss: 0.3762 - loss: 0.3226
Main loss: 0.31299731135368347
Aux loss: 0.3712167739868164
Main RMSE: 0.5599507689476013
Aux RMSE: 0.6095046401023865


In [ ]:
print("weighted_sum_of_losses", weighted_sum_of_losses)

weighted_sum_of_losses 0.31933990120887756


In [ ]:
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step


In [ ]:
print("y_pred_main", y_pred_main)
print("y_pred_aux", y_pred_aux)

y_pred_main [[0.38665903]
 [1.098732  ]
 [3.877286  ]]
y_pred_aux [[0.53755784]
 [0.9178368 ]
 [3.7925868 ]]


In [ ]:
y_test[:3]

array([0.477  , 0.458  , 5.00001])

은닉층의 검증을 위해 보조출력을 만들었지만 **은닉층이 학습을 잘 못 하고 있다**는 것을 알 수 있음
- 유용한 정보를 충분히 추출하지 못하고 있다
- 보조 출력이 역할을 잘 못하고 있기 때문에 주 출력 또한 유용한 학습을 하지 못하는 것을 알 수 있다.
**=> 하이퍼파라미터 튜닝이 필요함!**

In [ ]:
y_pred_tuple = model.predict((X_new_wide, X_new_deep))
y_pred = dict(zip(model.metrics_names, y_pred_tuple))
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
{'loss': array([[0.38665903],
       [1.098732  ],
       [3.877286  ]], dtype=float32), 'compile_metrics': array([[0.53755784],
       [0.9178368 ],
       [3.7925868 ]], dtype=float32)}


함수형 API 장점
- 자료형의 변환이 쉽다!
- 단일 입출력인 시퀀셜 API와 다르게 다중 입출력이 가능하다!